<a href="https://colab.research.google.com/github/Usnish-Mukherjee/Project-Intrusion-Detection-System/blob/main/Copy_of_multiclass_classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from glob import glob
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nd1 = np.zeros((0,79),dtype=object)
for x in glob('/content/drive/My Drive/Dataset/Data 1/*.csv'):
    print(x)
    nd1=np.vstack((nd1,pd.read_csv(x)))

/content/drive/My Drive/Dataset/Data 1/Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data 1/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data 1/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data 1/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data 1/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data 1/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data 1/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data 1/Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data 1/Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv


In [ ]:
nd1.shape

(8284195, 79)

In [ ]:
Dt1=nd1[:,:-1].astype(float)

In [ ]:
LNMV1=nd1[~np.isnan(Dt1).any(axis=1),-1]
DtNMV1=Dt1[~np.isnan(Dt1).any(axis=1)]
LNMIV1=LNMV1[~np.isinf(DtNMV1).any(axis=1)]
DtNMIV1=DtNMV1[~np.isinf(DtNMV1).any(axis=1)]

In [ ]:
del(DtNMV1)

In [ ]:
MCDt1=DtNMIV1[LNMIV1 !='Benign',:]

In [ ]:
MCDt1.shape

(2170743, 78)

In [ ]:
MCL1=LNMIV1[LNMIV1!='Benign']

In [ ]:
LNMIV1.shape,MCL1.shape

((8247888,), (2170743,))

In [ ]:
np.save('/content/drive/My Drive/Result/NMCx1', MinMaxScaler().fit_transform(MCDt1))
np.save('/content/drive/My Drive/Result/NMCy1', LabelEncoder().fit_transform(MCL1))

In [ ]:
nd2 = np.zeros((0,79),dtype=object)
nd2=np.vstack((nd2,pd.read_csv('/content/drive/My Drive/Dataset/Data 2/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv')))

In [ ]:
Dt2=nd2[:,:-1].astype(float)

In [ ]:
LNMV2=nd2[~np.isnan(Dt2).any(axis=1),-1]
DtNMV2=Dt2[~np.isnan(Dt2).any(axis=1)]
LNMIV2=LNMV2[~np.isinf(DtNMV2).any(axis=1)]
DtNMIV2=DtNMV2[~np.isinf(DtNMV2).any(axis=1)]

In [ ]:
del(DtNMV2)

In [ ]:
DtNMIV2.shape

(7889295, 78)

In [ ]:
MCDt2=DtNMIV2[LNMIV2 !='Benign',:]

In [ ]:
MCDt2.shape

(576191, 78)

In [ ]:
MCL2=LNMIV2[LNMIV2!='Benign']

In [ ]:
LNMIV2.shape,MCL2.shape

((7889295,), (576191,))

In [ ]:
np.save('/content/drive/My Drive/Result/NMCx2', MinMaxScaler().fit_transform(MCDt2))
np.save('/content/drive/My Drive/Result/NMCy2', LabelEncoder().fit_transform(MCL2))

In [3]:
import numpy as np
import os
import tensorflow as tf
import keras
from keras import models
from keras import layers

In [4]:
x1=np.load('/content/drive/My Drive/Result/NMCx1.npy')
y1=np.load('/content/drive/My Drive/Result/NMCy1.npy')
x2=np.load('/content/drive/My Drive/Result/NMCx2.npy')
y2=np.load('/content/drive/My Drive/Result/NMCy2.npy')
x=np.concatenate((x1,x2), axis=0)
y=np.concatenate((y1,y2), axis=0)
x=x.astype(np.float32)

In [5]:
x=np.delete(x,0,1)

In [6]:
(unique, counts) = np.unique(y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[     0 862382]
 [     1    611]
 [     2    230]
 [     3 686012]
 [     4   1730]
 [     5  41508]
 [     6 461912]
 [     7 139890]
 [     8  10990]
 [     9 193354]
 [    10 160639]
 [    11     87]
 [    12 187589]]


In [7]:
x.shape

(2746934, 77)

In [8]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=123)

In [9]:
from keras.utils import to_categorical
ztrain = to_categorical(ytrain, num_classes=None)
ztest = to_categorical(ytest, num_classes=None)

In [10]:
ztrain.shape

(2197547, 13)

**ANN with 5 hidden layer but different number of nodes**

In [11]:
def create_model():
  model = models.Sequential()
  model.add(layers.Dense(256,input_dim=x.shape[1],activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(128,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(64,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(32,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(16,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(np.unique(y).shape[0],activation='softmax',use_bias=True)) 
  return model

from keras.optimizers import * 
model = create_model()
opt = Adam(learning_rate=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['categorical_accuracy'])
model.summary()
history=model.fit(xtrain , ztrain ,batch_size=256 , epochs=75 ,verbose=1 ,shuffle=True)
result = model.evaluate(xtest,ztest)
dict(zip(model.metrics_names, result))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               19968     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2

{'categorical_accuracy': 0.9659183621406555, 'loss': 0.10087301582098007}

In [12]:
pred=model.predict(xtest)
y_pred = np.argmax(pred, axis=-1)

In [13]:
from sklearn import metrics
print("The accuracy score is",metrics.accuracy_score(ytest, y_pred))
print("The precision score is",metrics.precision_score(ytest, y_pred,average='weighted'))
print("The recall score is",metrics.recall_score(ytest, y_pred,average='weighted'))
print("The f1-score is",metrics.f1_score(ytest, y_pred,average='weighted'))
print('\n')
print(metrics.confusion_matrix(ytest, y_pred))
print('\n')
print("=== Classification Report ===")
print(metrics.classification_report(ytest, y_pred))
print('\n')

The accuracy score is 0.9659183781196133


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The precision score is 0.9662851015975095
The recall score is 0.9659183781196133
The f1-score is 0.9642811633377022


[[172561      0      0      0      0      0      0      0      0      0
      13      0      0]
 [    11      0      0      0      0      0      0      0      0      0
     112      0      0]
 [    11      0      0      0      0      0      0      0      0      0
      31      0      0]
 [     0      0      0 137533      0      0      0      0      0      0
       0      0      0]
 [     0      0      0      0    348      0      0      0      0      0
       0      0      0]
 [     0      0      0      0      0   8361      1      0      1      0
       0      0      0]
 [     0      0      0      0      0      3  92292      0      0      0
       0      0      0]
 [     0      0      0      0      0      0      0  14274      0  13639
       0      0      0]
 [     0      0      0      0      0      0      0      0   2207      0
      25      0      0]
 [     0      0   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    172574
           1       0.00      0.00      0.00       123
           2       0.00      0.00      0.00        42
           3       1.00      1.00      1.00    137533
           4       1.00      1.00      1.00       348
           5       1.00      1.00      1.00      8363
           6       1.00      1.00      1.00     92295
           7       0.76      0.51      0.61     27913
           8       1.00      0.99      0.99      2232
           9       0.71      0.88      0.79     38516
          10       0.99      0.99      0.99     32050
          11       0.00      0.00      0.00        25
          12       1.00      1.00      1.00     37373

    accuracy                           0.97    549387
   macro avg       0.73      0.72      0.72    549387
weighted avg       0.97      0.97      0.96    549387





**ANN with 3 Hidden layer**

In [ ]:
def create_model():
  model = models.Sequential()
  model.add(layers.Dense(128,input_dim=x.shape[1],activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(64,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(32,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(np.unique(y).shape[0],activation='softmax',use_bias=True)) 
  return model

from keras.optimizers import * 
model = create_model()
opt = Adam(learning_rate=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['categorical_accuracy'])
model.summary()
history=model.fit(xtrain , ztrain ,batch_size=256 , epochs=75 ,verbose=1 ,shuffle=True)
result = model.evaluate(xtest,ztest)
dict(zip(model.metrics_names, result))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               9984      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 13)                4

{'categorical_accuracy': 0.9659165740013123, 'loss': 0.06451693177223206}

In [ ]:
pred=model.predict(xtest)
y_pred = np.argmax(pred, axis=-1)

In [ ]:
from sklearn import metrics
print("The accuracy score is",metrics.accuracy_score(ytest, y_pred))
print("The precision score is",metrics.precision_score(ytest, y_pred,average='weighted'))
print("The recall score is",metrics.recall_score(ytest, y_pred,average='weighted'))
print("The f1-score is",metrics.f1_score(ytest, y_pred,average='weighted'))
print('\n')
print(metrics.confusion_matrix(ytest, y_pred))
print('\n')
print("=== Classification Report ===")
print(metrics.classification_report(ytest, y_pred))
print('\n')

The accuracy score is 0.9659165579090878


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The precision score is 0.9661331547508837
The recall score is 0.9659165579090878
The f1-score is 0.964604555057474


[[172546      0      0      0      0      0      0      0      0      0
      28      0      0]
 [     8     78      0      0      0      0      0      0      0      0
      37      0      0]
 [    10     20      0      0      0      0      0      0      0      0
      12      0      0]
 [     0      0      0 137533      0      0      0      0      0      0
       0      0      0]
 [     0      0      0      0    348      0      0      0      0      0
       0      0      0]
 [     0      0      0      0      0   8353     10      0      0      0
       0      0      0]
 [     0      0      0      0      0      1  92294      0      0      0
       0      0      0]
 [     0      0      0      0      0      0      0  14652      0  13261
       0      0      0]
 [     0      0      0      0      0      0      0      0   2207      0
      25      0      0]
 [     0      0    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    172574
           1       0.71      0.63      0.67       123
           2       0.00      0.00      0.00        42
           3       1.00      1.00      1.00    137533
           4       1.00      1.00      1.00       348
           5       1.00      1.00      1.00      8363
           6       1.00      1.00      1.00     92295
           7       0.74      0.52      0.61     27913
           8       1.00      0.99      0.99      2232
           9       0.72      0.87      0.78     38516
          10       1.00      1.00      1.00     32050
          11       0.00      0.00      0.00        25
          12       1.00      1.00      1.00     37373

    accuracy                           0.97    549387
   macro avg       0.78      0.77      0.77    549387
weighted avg       0.97      0.97      0.96    549387





**Ann with 5 hidden layer**  

---




In [ ]:
def create_model():
  model = models.Sequential()
  model.add(layers.Dense(256,input_dim=x.shape[1],activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(128,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(64,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(32,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(32,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(np.unique(y).shape[0],activation='softmax',use_bias=True)) 
  return model

from keras.optimizers import * 
model = create_model()
opt = Adam(learning_rate=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['categorical_accuracy'])
model.summary()
history=model.fit(xtrain , ztrain ,batch_size=256 , epochs=75 ,verbose=1 ,shuffle=True)
result = model.evaluate(xtest,ztest)
dict(zip(model.metrics_names, result))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 256)               19968     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)               

{'categorical_accuracy': 0.9662842154502869, 'loss': 0.06366468966007233}

In [ ]:
pred=model.predict(xtest)
y_pred = np.argmax(pred, axis=-1)

In [ ]:
from sklearn import metrics
print("The accuracy score is",metrics.accuracy_score(ytest, y_pred))
print("The precision score is",metrics.precision_score(ytest, y_pred,average='weighted'))
print("The recall score is",metrics.recall_score(ytest, y_pred,average='weighted'))
print("The f1-score is",metrics.f1_score(ytest, y_pred,average='weighted'))
print('\n')
print(metrics.confusion_matrix(ytest, y_pred))
print('\n')
print("=== Classification Report ===")
print(metrics.classification_report(ytest, y_pred))
print('\n')

The accuracy score is 0.9662842404352487


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The precision score is 0.9669039060967618
The recall score is 0.9662842404352487
The f1-score is 0.964728293060669


[[172546      0      0      0      0      0      0      0      0      0
      28      0      0]
 [     7     78      0      0      0      0      0      0      0      0
      38      0      0]
 [    10     20      0      0      0      0      0      0      0      0
      12      0      0]
 [     0      0      0 137533      0      0      0      0      0      0
       0      0      0]
 [     0      0      0      0    348      0      0      0      0      0
       0      0      0]
 [     0      0      0      0      0   8357      4      0      2      0
       0      0      0]
 [     0      0      0      0      0     16  92279      0      0      0
       0      0      0]
 [     0      0      0      0      0      0      0  14214      0  13699
       0      0      0]
 [     0      0      0      0      0      0      0      0   2206      0
      26      0      0]
 [     0      0    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    172574
           1       0.70      0.63      0.66       123
           2       0.00      0.00      0.00        42
           3       1.00      1.00      1.00    137533
           4       1.00      1.00      1.00       348
           5       1.00      1.00      1.00      8363
           6       1.00      1.00      1.00     92295
           7       0.76      0.51      0.61     27913
           8       1.00      0.99      0.99      2232
           9       0.71      0.88      0.79     38516
          10       1.00      1.00      1.00     32050
          11       0.00      0.00      0.00        25
          12       1.00      1.00      1.00     37373

    accuracy                           0.97    549387
   macro avg       0.78      0.77      0.77    549387
weighted avg       0.97      0.97      0.96    549387





**Ann with 4 hidden Layer**

---






In [ ]:
def create_model():
  model = models.Sequential()
  model.add(layers.Dense(256,input_dim=x.shape[1],activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(128,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(64,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(32,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(np.unique(y).shape[0],activation='softmax',use_bias=True)) 
  return model

from keras.optimizers import * 
model = create_model()
opt = Adam(learning_rate=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['categorical_accuracy'])
model.summary()
history=model.fit(xtrain , ztrain ,batch_size=256 , epochs=100 ,verbose=1 ,shuffle=True)
result = model.evaluate(xtest,ztest)
dict(zip(model.metrics_names, result))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               19968     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2

{'categorical_accuracy': 0.9669540524482727, 'loss': 0.06335224956274033}

In [ ]:
pred=model.predict(xtest)
y_pred = np.argmax(pred, axis=-1)

In [ ]:
from sklearn import metrics
print("The accuracy score is",metrics.accuracy_score(ytest, y_pred))
print("The precision score is",metrics.precision_score(ytest, y_pred,average='weighted'))
print("The recall score is",metrics.recall_score(ytest, y_pred,average='weighted'))
print("The f1-score is",metrics.f1_score(ytest, y_pred,average='weighted'))
print('\n')
print(metrics.confusion_matrix(ytest, y_pred))
print('\n')
print("=== Classification Report ===")
print(metrics.classification_report(ytest, y_pred))
print('\n')

The accuracy score is 0.966954077908651


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The precision score is 0.9677786887141197
The recall score is 0.966954077908651
The f1-score is 0.965423690585904


[[172545      0      0      0      0      0      0      0      0      0
      29      0      0]
 [     7     67      0      0      0      0      0      0      0      0
      49      0      0]
 [    10      2     18      0      0      0      0      0      0      0
      12      0      0]
 [     0      0      0 137533      0      0      0      0      0      0
       0      0      0]
 [     0      0      0      0    348      0      0      0      0      0
       0      0      0]
 [     0      0      0      0      0   8360      3      0      0      0
       0      0      0]
 [     0      0      0      0      0      0  92295      0      0      0
       0      0      0]
 [     0      0      0      0      0      0      0  14345      0  13568
       0      0      0]
 [     0      0      0      0      0      0      0      0   2206      0
      26      0      0]
 [     0      0     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    172574
           1       0.97      0.54      0.70       123
           2       0.95      0.43      0.59        42
           3       1.00      1.00      1.00    137533
           4       1.00      1.00      1.00       348
           5       1.00      1.00      1.00      8363
           6       1.00      1.00      1.00     92295
           7       0.77      0.51      0.62     27913
           8       1.00      0.99      0.99      2232
           9       0.72      0.89      0.79     38516
          10       1.00      1.00      1.00     32050
          11       0.00      0.00      0.00        25
          12       1.00      1.00      1.00     37373

    accuracy                           0.97    549387
   macro avg       0.88      0.80      0.82    549387
weighted avg       0.97      0.97      0.97    549387





In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc_history=rfc.fit(xtrain,ytrain)
rfc_predict = rfc.predict(xtest)

In [ ]:
from sklearn import metrics
print("The accuracy score is",metrics.accuracy_score(ytest, rfc_predict.round()))
print("The precision score is",metrics.precision_score(ytest, rfc_predict.round(),average='weighted'))
print("The recall score is",metrics.recall_score(ytest, rfc_predict.round(),average='weighted'))
print("The f1-score is",metrics.f1_score(ytest, rfc_predict.round(),average='weighted'))
print('\n')
print(metrics.confusion_matrix(ytest, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(metrics.classification_report(ytest, rfc_predict))
print('\n')

The accuracy score is 0.966881269487629
The precision score is 0.96763677687889
The recall score is 0.966881269487629
The f1-score is 0.9654394758618212


[[172565      0      0      0      0      0      0      0      0      0
       9      0      0]
 [     6     81      0      0      0      0      0      0      0      0
      35      1      0]
 [     7      1     29      0      0      0      0      0      0      0
       4      1      0]
 [     0      0      0 137497      0      0      0      0      0      0
      36      0      0]
 [     0      0      0      0    348      0      0      0      0      0
       0      0      0]
 [     0      0      0      0      0   8362      1      0      0      0
       0      0      0]
 [     0      0      0      0      0      0  92295      0      0      0
       0      0      0]
 [     0      0      0      0      0      0      0  14433      0  13480
       0      0      0]
 [     0      0      0      0      0      0      0      0   2196      0
     